In [1]:
# 分析标注结果，包括标注的覆盖率、acc；标注员之间的一致性

In [26]:
from sklearn.metrics import cohen_kappa_score, accuracy_score
import json
import numpy as np

In [42]:
DATASET = 'tnews'
split = 'train'
compare_model = ['paddle_nlp', 'gt'] #[chatglm2-6b, qwen-7b-chat, siamese_uninlu, paddle_nlp, gt(需要放到第二个位置）]

In [43]:
def get_anno(dataset, model_name, split='train'):
    res = {}
    if model_name == 'gt':
        fname = f'/mnt/workspace/exp_dataset/{dataset}/{split}.json'
    else:
        fname = f'/mnt/workspace/exp_dataset/{dataset}/result_{model_name}_{dataset}_{split}.json'
    with open(fname, 'r', encoding='utf-8') as f:
        for line in f:
            line = json.loads(line.strip())
            res[line['sentence']] = line['label']
    return res
            

In [ ]:
anno1 = get_anno(DATASET, compare_model[0], split=split)
anno2 = get_anno(DATASET, compare_model[1], split=split)

intersect_texts = list(set(anno1.keys()) & set(anno2.keys()))
union_texts = list(set(anno1.keys()) | set(anno2.keys()))

anno1_labels, anno2_labels = [], []
for text in intersect_texts:
    anno1_labels.append(anno1[text])
    anno2_labels.append(anno2[text])

if 'gt' == compare_model[1]:
    # 输出覆盖率，准确率
    print(f'标注员{compare_model[0]}的覆盖率:{len(intersect_texts)/len(union_texts)}, Acc:{accuracy_score(np.array(anno2_labels), np.array(anno1_labels))}')
else:
    print(f'标注员1:{compare_model[0]}, 共标注结果{len(anno1)}条')
    print(f'标注员2:{compare_model[1]}, 共标注结果{len(anno2)}条')
    print(f'两个标注员的交集一共{len(intersect_texts)}条数据')
    print(f'两个标注员的jaccard系数为{len(intersect_texts)/len(union_texts)}')
    print(f'两个标注员的cohen kappa score = {cohen_kappa_score(anno1_labels, anno2_labels)}')
